# Count MHWEs

In [1]:
# imports
import os
import numpy as np
import datetime

import pandas

from oceanpy.sst import utils as sst_utils

from mhw_analysis.systems import io as mhw_sys_io

# Load up MHWS

In [2]:
#mask_Id=1475052 # 2019, local
#vary=True
mask_Id = 1458524 # 2019
vary=False
mhw_sys_file= os.path.join(os.getenv('MHW'), 'db', 'MHWS_2019.csv')
mask_file=os.path.join(os.getenv('MHW'), 'db', 'MHWS_2019_mask.nc')

# Load MHW Systems
mhw_sys = mhw_sys_io.load_systems(mhw_sys_file=mhw_sys_file, 
                                      vary=vary)

Loading systems from /data/Projects/Oceanography/MHW/db/MHWS_2019.csv
Done


In [3]:
idx = np.where(mhw_sys.mask_Id == mask_Id)[0][0]
isys = mhw_sys.iloc[idx]
sys_startdate = isys.datetime - datetime.timedelta(days=int(isys.zcen)-int(isys.zboxmin))

# Grab the mask
mask_da = mhw_sys_io.load_mask_from_system(isys, vary=vary, mhw_mask_file=mask_file)

Loading mask from system: 
 Id                                 414252
NVox                               752073
NVox_km                       516746800.0
category                                2
mask_Id                           1458524
max_area                             9086
max_area_km                     6500934.0
xcen                            464.78653
xboxmin                               390
xboxmax                               556
ycen                            867.93195
yboxmin                               611
yboxmax                              1018
zcen                            5177.9043
zboxmin                              5046
zboxmax                              5352
date                           1996-03-05
lat                             26.321632
lon                            217.107986
duration                307 days 00:00:00
datetime              1996-03-05 00:00:00
startdate      1995-10-25 02:17:48.480000
enddate        1996-08-27 02:17:48.480000
Name: 

In [4]:
mask_da.shape

(720, 1440, 307)

## Find spatial boundaries

In [5]:
lat_coord, lon_coord = sst_utils.noaa_oi_coords()

In [6]:
lat_min, lat_max = lat_coord[isys.xboxmin], lat_coord[isys.xboxmax]
lat_min, lat_max

(7.625, 49.125)

In [7]:
lon_min, lon_max = lon_coord[isys.yboxmin], lon_coord[isys.yboxmax]
lon_min, lon_max

(152.875, 254.625)

# MHWEs

## Load

In [8]:
mhwe_file = os.path.join(os.getenv('MHW'), 'db', 'mhw_events_allsky_2019_local.parquet')

In [9]:
mhwe = pandas.read_parquet(mhwe_file)

In [10]:
mhwe.head()

,index,time_start,time_end,time_peak,duration,duration_moderate,duration_strong,duration_severe,duration_extreme,category,...,intensity_max_relThresh,intensity_max_abs,intensity_mean_relThresh,intensity_mean_abs,intensity_var_relThresh,intensity_var_abs,intensity_cumulative_relThresh,intensity_cumulative_abs,rate_onset,rate_decline
0,0,723911,723931,723924,21,14,7,0,0,1,...,0.433648,-0.457613,0.217017,-0.696718,0.124336,0.144548,4.557350,-14.631067,0.014541,0.066994
1,1,725746,725754,725752,9,9,0,0,0,0,...,0.074059,-0.803030,0.040143,-0.851957,0.021723,0.033850,0.361283,-7.667612,0.015423,0.027143
2,2,726840,726890,726857,51,51,0,0,0,0,...,0.259864,-0.582125,0.098204,-0.766680,0.073886,0.096659,5.008395,-39.100677,0.017261,0.006991
3,3,727139,727277,727210,139,39,46,8,46,3,...,1.723271,0.824553,0.597275,-0.426060,0.540468,0.667796,83.021271,-59.222404,0.026428,0.025159
4,4,727493,727577,727556,85,35,37,13,0,2,...,0.345204,-0.678901,0.173176,-1.002005,0.081395,0.198490,14.719926,-85.170387,0.008077,0.015180


## Spatial cut

In [11]:
gd_lat = (mhwe.lat >= lat_min) & (mhwe.lat <=lat_max)
gd_lon = (mhwe.lon >= lon_min) & (mhwe.lon <=lon_max)

## Time cut

In [12]:
mhws_ts_ord = isys.startdate.toordinal()
mhws_te_ord = isys.enddate.toordinal()
mhws_ts_ord, mhws_te_ord

(728591, 728898)

In [13]:
gd_time = (mhwe.time_start < mhws_te_ord) & (mhwe.time_end > mhws_ts_ord)

In [14]:
np.sum(gd_lat & gd_lon & gd_time)

162092

## Cut

In [15]:
cut_mhwe = mhwe[gd_lat & gd_lon & gd_time].copy()

In [16]:
cut_mhwe[['lat', 'lon', 'time_start']].head()

,lat,lon,time_start
18247432,7.625,152.875,728605
18247433,7.625,152.875,728897
18247498,7.875,152.875,728605
18247499,7.875,152.875,728897
18247634,8.375,152.875,728672


## Test case

In [17]:
imhwe = cut_mhwe.iloc[0]
imhwe

index                                         1967257
time_start                                     728605
time_end                                       728610
time_peak                                      728608
duration                                            6
duration_moderate                                   5
duration_strong                                     1
duration_severe                                     0
duration_extreme                                    0
category                                            1
n_events                                            0
ievent                                             35
date                              1995-11-08 00:00:00
lat                                             7.625
lon                                           152.875
intensity_max                                1.233524
intensity_mean                               0.913802
intensity_var                                0.203745
intensity_cumulative        

In [18]:
x_i = int((imhwe.lat + 89.875)/0.25)
x_i

390

In [19]:
y_i = int((imhwe.lon - 0.125)/0.25)
y_i

611

In [20]:
mhws_ts_ord

728591

In [21]:
t_i = (imhwe.time_start-mhws_ts_ord) + 2 # for boundary
t_i

16

### Test the mask

In [22]:
mask_da.data[x_i, y_i, t_i-3:t_i+4]

array([1454849, 1454849, 1454849, 1454849, 1454849,       0,       0],
      dtype=int32)

## Loop em!

In [23]:
count = 0
max_dur = 0
for ss in range(len(cut_mhwe)):
    imhwe = cut_mhwe.iloc[ss]
    x_i = int((imhwe.lat + 89.875)/0.25)
    y_i = int((imhwe.lon - 0.125)/0.25)
    t_i = (imhwe.time_start-mhws_ts_ord)  # for boundary
    #
    try:
        if mask_da.data[x_i, y_i, t_i] == isys.mask_Id:
            #print(ss)
            count += 1
            if imhwe.duration > max_dur:
                max_dur = imhwe.duration
    except:
        import pdb; pdb.set_trace()

#
print(f"Final count = {count}!")
print(f"Longest MHWE = {max_dur} days")
#    if (ss % 100) == 0:
#        import pdb; pdb.set_trace()

Final count = 44065!
Longest MHWE = 133 days
